In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# 웹드라이버 실행
# url = 'https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=474&sub_category_id=474&sort=200'  # 상의
url = 'https://zigzag.kr/categories/-1?title=%EC%95%84%EC%9A%B0%ED%84%B0&category_id=-1&middle_category_id=436&sort=200'  # 아우터
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)  # 페이지 로딩 대기

# 스크롤하여 데이터 수집
scrolls = 9  # 원하는 만큼 스크롤 (조정 가능)
collected_links = set()  # 중복 방지를 위한 set()

for _ in range(scrolls):
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # 페이지가 로딩될 시간을 줌
    
    # 페이지 HTML 업데이트
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # 상품 링크 수집 (새로운 상품이 로딩되었을 가능성이 있으므로 계속 추가)
    links = soup.select("#__next > div.zds-themes.light-theme > div.css-xrh5h3.edi339y0 > main > section.css-j2fe9p.eeob9mf0 > div a")
    
    for link in links:
        product_url = link.get('href')
        if product_url and product_url.startswith('/catalog/products/'):
            collected_links.add(f'https://zigzag.kr{product_url}')  # 중복 제거 위해 set에 추가

# 최종 수집된 링크 확인
print(f"총 {len(collected_links)}개의 상품 링크를 수집했습니다.")

# 수집된 정보를 저장할 리스트
data = []

# 상품 상세 정보 크롤링
for idx, full_url in enumerate(list(collected_links)):  # 모든 상품 링크를 크롤링
    driver.get(full_url)
    time.sleep(2)
    
    item_page = BeautifulSoup(driver.page_source, 'html.parser')
    
    try:
        brand_name = item_page.select_one('.css-gwr30y').text.strip()
        item_name = item_page.select_one('h1.css-1n8byw.e14n6e5u1').text.strip()
        origin_price = item_page.select_one('.css-14j45be.e1yx2lle2').text.strip()
        discount_price = item_page.select_one('.css-no59fe.e1ovj4ty1').text.strip()
        review_count = item_page.select_one('span.css-1ovjo5n').text.strip()
        item_score_avg = item_page.select_one('.css-1hld56p.e71452m2').text.strip()
        brand_keyword = [span.text for span in item_page.select("#__next > div.zds-themes.light-theme > div.css-xrh5h3.edi339y0 > div > div.shop_row > div > div > div > div > div:nth-child(2) > div.css-2jjng0.eq4fxvi7 span")]
        
        print(brand_name, item_name, origin_price, discount_price, review_count, item_score_avg, brand_keyword)
        # 수집한 데이터를 리스트에 추가
        data.append({
            '브랜드': brand_name,
            '상품명': item_name,
            '원가': origin_price,
            '할인 가격': discount_price,
            '리뷰 개수': review_count,
            '평점': item_score_avg,
            '키워드': ', '.join(brand_keyword)
        })
        
        print(len(data))

    except AttributeError:
        print(f"[{idx+1}] 데이터가 부족한 상품입니다. URL: {full_url}")
    
    time.sleep(2)

# 크롤링된 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)

# DataFrame을 CSV 파일로 저장
df.to_csv(r'D:\0_Yebang\취업\포트폴리오\개인프로젝트\Shopping_mall_analyze\data\outer_info.csv', index=False, encoding='utf-8')

# 브라우저 종료
driver.quit()

print(f"총 {len(df)}개의 상품 정보를 CSV로 저장했습니다.")